### Pedestrian Traffic Simulation

**Overview**
This project simulates pedestrian movement through a crosswalk, incorporating a traffic light system that alternates between red and green. The simulation tracks pedestrians as they walk, wait at the crosswalk, cross the road, and finish their journey.

**Features**
* Traffic Light System: Simulates a traffic light alternating between red and green. Pedestrians wait at the crosswalk if the light is red.
* Randomized Pedestrian Behavior: Pedestrians have random arrival times and varying walking speeds, simulating real-life behavior.
* Real-time Visualization: An animated plot visualizes the pedestrian movements through the different stages of their journey.

**Code Structure**
* TrafficLight Class: Manages the state of the traffic light (green/red) and signals to pedestrians when they can cross.
* Pedestrian Function: Simulates the behavior of a pedestrian, including walking, waiting at the crosswalk, crossing, and finishing.
* Pedestrian Generator: Generates pedestrians at random intervals to simulate continuous pedestrian traffic.
* Simulation Function: Runs the entire simulation and returns the data for visualization.
* Visualization: The simulation results are visualized using matplotlib.animation.FuncAnimation, showing the progression of each pedestrian.

**Customization**
* Number of Pedestrians: Modify the NUM_PEDESTRIANS variable to simulate more or fewer pedestrians.
* Traffic Light Timing: Adjust GREEN_LIGHT_DURATION and RED_LIGHT_DURATION to change how long the traffic light stays green or red.
* Simulation Time: Change SIMULATION_TIME to simulate for a longer or shorter period.


In [7]:
import simpy
import random
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# Parameters
NUM_PEDESTRIANS = 10 
ARRIVAL_INTERVAL = 5  
WALKING_TIME = (1, 3)  
CROSSWALK_TIME = (1, 2)  
SIMULATION_TIME = 50  
GREEN_LIGHT_DURATION = 10  
RED_LIGHT_DURATION = 5  

**Parameters:**
1.	NUM_PEDESTRIANS:  This parameter defines the total number of pedestrians that will be generated during the simulation.
2.	ARRIVAL_INTERVAL:  This is the average time interval (in minutes) between the arrival of each pedestrian at the crosswalk. The actual time between arrivals is randomly determined using an exponential distribution with this average.
3.	WALKING_TIME:  This defines the range (in minutes) of time it takes for a pedestrian to walk between points. The walking time is randomly selected from within this range.
4.	CROSSWALK_TIME:  This parameter defines the range (in minutes) of time a pedestrian has to wait at the crosswalk before they can cross. The wait time is randomly selected from within this range.
5.	SIMULATION_TIME:  This defines the total duration (in minutes) of the entire simulation. Once this time is reached, the simulation stops, even if all pedestrians haven't finished their journey.
6.	GREEN_LIGHT_DURATION:  This defines how long (in minutes) the traffic light stays green, allowing pedestrians to cross the road.
7.	RED_LIGHT_DURATION: This defines how long (in minutes) the traffic light stays red, preventing pedestrians from crossing the road.


In [8]:
# Define the traffic light class

class TrafficLight:
    def __init__(self, env): 
        self.env = env
        self.color = 'GREEN'
        self.green = env.event()  # Event to signal the green light
        self.process = env.process(self.run())

    def run(self):
        while True:
            self.color = 'GREEN'
            self.green.succeed()  # Signal the green light
            self.green = self.env.event()  # Reset the event for the next green phase
            yield self.env.timeout(GREEN_LIGHT_DURATION)

            self.color = 'RED'
            yield self.env.timeout(RED_LIGHT_DURATION)

# Define the pedestrian simulation
def pedestrian(env, name, light):
    """Simulates a pedestrian's movement through the system."""
    print(f'{name} starts walking at {env.now:.2f}')
    start_time = env.now
    pedestrian_positions.append((start_time, name, 0))
    
    # Walk to crosswalk
    walking_time = random.uniform(*WALKING_TIME)
    yield env.timeout(walking_time)
    reach_crosswalk_time = env.now
    pedestrian_positions.append((reach_crosswalk_time, name, 1))
    print(f'{name} reaches crosswalk at {env.now:.2f}')
    
    # Wait at crosswalk if light is red
    if light.color == 'RED':
        print(f'{name} waits at crosswalk at {env.now:.2f}')
        yield light.green  # Wait for the green light
    
    cross_time = env.now
    pedestrian_positions.append((cross_time, name, 2))
    print(f'{name} crosses at {env.now:.2f}')
    
    # Continue walking
    walking_time = random.uniform(*WALKING_TIME)
    yield env.timeout(walking_time)
    finish_time = env.now
    pedestrian_positions.append((finish_time, name, 3))
    print(f'{name} finishes walking at {env.now:.2f}')

# Generate pedestrians
def pedestrian_generator(env, light):
    """Generates pedestrians at random intervals."""
    for i in range(NUM_PEDESTRIANS):
        yield env.timeout(random.expovariate(1.0 / ARRIVAL_INTERVAL))
        env.process(pedestrian(env, f'Pedestrian {i+1}', light))

# Simulate the pedestrian traffic
def simulate_pedestrian_traffic():
    global pedestrian_positions
    pedestrian_positions = []
    env = simpy.Environment()
    light = TrafficLight(env)
    env.process(pedestrian_generator(env, light))
    env.run(until=SIMULATION_TIME)
    return pd.DataFrame(pedestrian_positions, columns=["Time", "Pedestrian", "Event"])

In [25]:
# Creating a Dataframe to store the simulation results
df = simulate_pedestrian_traffic()

Pedestrian 1 starts walking at 6.24
Pedestrian 2 starts walking at 6.29
Pedestrian 1 reaches crosswalk at 8.24
Pedestrian 1 crosses at 8.24
Pedestrian 2 reaches crosswalk at 8.28
Pedestrian 2 crosses at 8.28
Pedestrian 1 finishes walking at 9.61
Pedestrian 2 finishes walking at 9.89
Pedestrian 3 starts walking at 14.96
Pedestrian 3 reaches crosswalk at 17.70
Pedestrian 3 crosses at 17.70
Pedestrian 3 finishes walking at 20.23
Pedestrian 4 starts walking at 21.90
Pedestrian 4 reaches crosswalk at 23.11
Pedestrian 4 crosses at 23.11
Pedestrian 4 finishes walking at 24.86
Pedestrian 5 starts walking at 27.86
Pedestrian 5 reaches crosswalk at 29.60
Pedestrian 5 waits at crosswalk at 29.60
Pedestrian 6 starts walking at 29.96
Pedestrian 5 crosses at 30.00
Pedestrian 5 finishes walking at 31.91
Pedestrian 6 reaches crosswalk at 32.06
Pedestrian 6 crosses at 32.06
Pedestrian 6 finishes walking at 34.96
Pedestrian 7 starts walking at 35.61
Pedestrian 7 reaches crosswalk at 37.38
Pedestrian 7 c